![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [20]:
from AlgorithmImports import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import plotly.graph_objects as go
from datetime import datetime


# --- 1. 初始化和历史数据请求 ---
qb = QuantBook()
symbol = qb.add_future(Futures.Metals.GOLD, Resolution.MINUTE).symbol


history_date = datetime(2025, 11, 18, 21, 0, 0)

df_history = qb.history(symbol, history_date, history_date + timedelta(days=2),
                        resolution=Resolution.MINUTE,
                        extended_market_hours=True,
                        data_mapping_mode=DataMappingMode.OPEN_INTEREST_ANNUAL, # 数据映射模式，这个会根据交易量切换到当年后续更大的合约，Warning, 但正确性有待验证
                        data_normalization_mode=DataNormalizationMode.RAW, # 数据连续模式，ATAS是RAW, tradingview 是BACKWARDS_RATIO，能够使得连续。注意，实盘需要使用当期合约数据
                        fill_forward=True
                        ).reset_index()
ChangedEvent = qb.history(SymbolChangedEvent, symbol,history_date, history_date+ timedelta(days=1))
print(len(df_history))


candlestick_c0 = go.Candlestick(
    x=df_history['time'], 
    open=df_history['open'], 
    high=df_history['high'], 
    low=df_history['low'], 
    close=df_history['close'],
    name='C0 Candlestick'
)

# 定义图表布局 (C0)
layout_c0 = go.Layout(
    title=f'C0 (Front Month) Candlestick Chart: {symbol.Value}',
    xaxis=dict(title='Date (HKT)'),
    yaxis=dict(title='Price (USD)'),
    xaxis_rangeslider_visible=False
)

# 创建并显示图表 (C0)
fig_c0 = go.Figure(data=[candlestick_c0], layout=layout_c0)
fig_c0.show()
